This was a project from the Coursera coarse 'ML Algorithms' a part of the 'Exam Prep MLS-C01: AWS Certified Specialty Machine Learning'track, the project is from 2020 so I needed to ,ake extensive modifications to get it worling in todays SageMaker.

In [1]:
import numpy as np
import boto3
import sagemaker
import io
import sagemaker.amazon.common as smac
import os
import pandas as pd
from io import StringIO
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Initialize variables
bucket = '<your s3 bucket goes here>'
object_key = 'day.csv'
prefix = 'realestate'
train_key = 'train/linearregression'
output_key = 'output/linearregression'

In [3]:
# Load data from S3
s3 = boto3.resource('s3')
csv_obj = s3.Object(bucket, object_key)
csv_string = csv_obj.get()['Body'].read().decode('utf-8')
df = pd.read_csv(StringIO(csv_string))

In [4]:
# Data preprocessing
df['dteday'] = df['dteday'].str.replace("-", "")
df_shuffled = df.sample(frac=1, random_state=1729).reset_index(drop=True)
train_data, test_data = np.split(df_shuffled, [int(0.7 * len(df_shuffled))])

feature_dataset = train_data[['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday', 
                              'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
                              'casual', 'registered']]

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [5]:
features = np.array(feature_dataset.values).astype('float32')
label_dataset = train_data['cnt']
labels = np.array(label_dataset.values).astype('float32')
labels_vec = np.squeeze(np.asarray(labels))

# Prepare data for training
buffer = io.BytesIO()
smac.write_numpy_to_dense_tensor(buffer, features, labels_vec)
buffer.seek(0)

0

In [6]:
# Upload train data to S3
boto3.resource('s3').Bucket(bucket).Object(os.path.join(prefix, train_key)).upload_fileobj(buffer)
# Define S3 locations
s3_training_data_location = f's3://{bucket}/{prefix}/{train_key}'
output_location = f's3://{bucket}/{prefix}/{output_key}'

# Get SageMaker role
role = get_execution_role()
sagemaker_session = sagemaker.Session()

# Retrieve the container image URI
linear_container = sagemaker.image_uris.retrieve('linear-learner', boto3.Session().region_name)

In [7]:
# Define the estimator
linear = sagemaker.estimator.Estimator(
    linear_container,
    role=role,
    instance_count=1,
    instance_type='ml.c4.xlarge',
    output_path=output_location,
    sagemaker_session=sagemaker_session
)

# Set hyperparameters
linear.set_hyperparameters(
    feature_dim=15,
    mini_batch_size=4,
    predictor_type='regressor'
)

# Train the model
linear.fit({'train': s3_training_data_location})


INFO:sagemaker:Creating training-job with name: linear-learner-2024-07-22-18-37-20-842


2024-07-22 18:37:21 Starting - Starting the training job...
2024-07-22 18:37:38 Starting - Preparing the instances for training...
2024-07-22 18:38:00 Downloading - Downloading input data...
2024-07-22 18:38:35 Downloading - Downloading the training image.........
2024-07-22 18:40:01 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
[07/22/2024 18:40:08 INFO 139637672281920] Reading default configuration from /opt/amazon/lib/python3.8/site-packages/algorithm/resources/default-input.json: {'mini_batch_size': '1000', 'epochs': '15', 'feature_dim': 'auto', 'use_bias': 'true', 'binary_classifier_model_selection_criteria': 'accuracy', 'f_beta': '1.0', 'target_recall': '0.8', 'target_precision': '0.8', 'num_models': 'auto', 'num_calibration_samples': '10000000', 'init_method': 'uniform', 'init_scale': '0.07', 'init_sigma': '0.01', 'init_bias': '0.0', 'optimizer': 'auto', 'loss':

[2024-07-22 18:40:16.498] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 8, "duration": 1613, "num_examples": 128, "num_bytes": 53144}
#metrics {"StartTime": 1721673616.4985895, "EndTime": 1721673616.4986541, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 0}, "Metrics": {"train_mse_objective": {"sum": 0.0189645591867523, "count": 1, "min": 0.0189645591867523, "max": 0.0189645591867523}}}
#metrics {"StartTime": 1721673616.4987247, "EndTime": 1721673616.4987376, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 1}, "Metrics": {"train_mse_objective": {"sum": 0.022094634444980814, "count": 1, "min": 0.022094634444980814, "max": 0.022094634444980814}}}
#metrics {"StartTime": 1721673616.498771, "EndTime": 1721673616.4987788, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 2, "model": 

[2024-07-22 18:40:28.919] [tensorio] [info] epoch_stats={"data_pipeline": "/opt/ml/input/data/train", "epoch": 24, "duration": 1559, "num_examples": 128, "num_bytes": 53144}
#metrics {"StartTime": 1721673628.9190958, "EndTime": 1721673628.9191542, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 0}, "Metrics": {"train_mse_objective": {"sum": 0.00016004641450516235, "count": 1, "min": 0.00016004641450516235, "max": 0.00016004641450516235}}}
#metrics {"StartTime": 1721673628.9192235, "EndTime": 1721673628.9192336, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "epoch": 10, "model": 1}, "Metrics": {"train_mse_objective": {"sum": 4.185531987944367e-05, "count": 1, "min": 4.185531987944367e-05, "max": 4.185531987944367e-05}}}
#metrics {"StartTime": 1721673628.919263, "EndTime": 1721673628.9192693, "Dimensions": {"Algorithm": "Linear Learner", "Host": "algo-1", "Operation": "training", "ep


2024-07-22 18:40:41 Uploading - Uploading generated training model
2024-07-22 18:40:55 Completed - Training job completed
Training seconds: 175
Billable seconds: 175


In [8]:
# Deploy the model
linear_predictor = linear.deploy(initial_instance_count=1,
                                 instance_type='ml.c4.xlarge',
                                 endpoint_name='bikeshare-sagemaker-regression-v02')


INFO:sagemaker:Creating model with name: linear-learner-2024-07-22-18-48-04-151
INFO:sagemaker:Creating endpoint-config with name bikeshare-sagemaker-regression-v02
INFO:sagemaker:Creating endpoint with name bikeshare-sagemaker-regression-v02


---------!

In [12]:
# Get predictions using test data
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import JSONDeserializer

# Set the content type and serializers for the predictor
linear_predictor.content_type = 'text/csv'
linear_predictor.serializer = CSVSerializer()
linear_predictor.deserializer = JSONDeserializer()


In [14]:
# Prepare test feature dataset
test_feature_dataset = test_data[['instant', 'dteday', 'season', 'yr', 'mnth', 'holiday', 'weekday', 
                                  'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed',
                                  'casual', 'registered']]
# Prepare actual values
test_actuals = np.array(test_data['cnt'].astype('float32'))

predictions = []
actuals = []

# Loop through test features and actual values to get predictions
for tf, actual in zip(test_feature_dataset.values, test_actuals):
    prediction = linear_predictor.predict(tf)
    predictions.append(prediction['predictions'][0]['score'])
    actuals.append(actual)
    print('prediction: ', prediction['predictions'][0]['score'], '\t\tactual:', str(actual))

prediction:  799.390625 		actual: 801.0
prediction:  5216.6875 		actual: 5217.0
prediction:  7765.5625 		actual: 7767.0
prediction:  6850.9375 		actual: 6852.0
prediction:  2209.96875 		actual: 2209.0
prediction:  6289.28125 		actual: 6290.0
prediction:  4790.703125 		actual: 4792.0
prediction:  1866.796875 		actual: 1865.0
prediction:  5670.375 		actual: 5668.0
prediction:  4492.25 		actual: 4492.0
prediction:  4368.203125 		actual: 4367.0
prediction:  2402.453125 		actual: 2402.0
prediction:  3847.546875 		actual: 3846.0
prediction:  4787.96875 		actual: 4788.0
prediction:  3189.015625 		actual: 3190.0
prediction:  3006.515625 		actual: 3005.0
prediction:  7284.625 		actual: 7286.0
prediction:  2131.234375 		actual: 2132.0
prediction:  5462.21875 		actual: 5464.0
prediction:  4984.09375 		actual: 4985.0
prediction:  6304.625 		actual: 6304.0
prediction:  5532.03125 		actual: 5532.0
prediction:  8008.171875 		actual: 8009.0
prediction:  3000.46875 		actual: 2999.0
prediction:  4337.14

prediction:  1890.6875 		actual: 1891.0
prediction:  3376.5625 		actual: 3376.0
prediction:  6977.078125 		actual: 6978.0
prediction:  6657.6875 		actual: 6660.0
prediction:  5112.625 		actual: 5115.0
prediction:  2162.65625 		actual: 2162.0
prediction:  5875.484375 		actual: 5875.0


In [15]:
# Get accuracy using cosine similarity method
from numpy import dot
from numpy.linalg import norm

tolerance = 1e-10  # Corrected the tolerance value
accuracy = (dot(actuals, predictions) / (norm(actuals) * norm(predictions))) * 100

# Check if the accuracy calculation is above the tolerance level
if abs(accuracy) < tolerance:
    accuracy = 0.0  # Set accuracy to 0 if it's below the tolerance
print('Accuracy:', accuracy)


Accuracy: 100.0000031367257


In [ ]:
# delete endpoint or alteratly this can be done in the SageMaker console
endpoint_name = '<your-endpoint-name>'
# Delete the endpoint
response = sagemaker_client.delete_endpoint(EndpointName=endpoint_name)
print(f"Endpoint {endpoint_name} deletion response: {response}")
